<h3>Install Bedrock and Langchain</h3>

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

<h3>Language Models</h3>

In [ ]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )


In [ ]:
response = textgen_llm("""
SystemMessage='you are a nice AI bot that helps users with recommendations',
HumanMessage='give me a list of cities to visit in Germany, add a brief description of each',
""")

print(response)

## Langchain Basics

<h3>Prompts</h3>

**Simple prompt**

In [ ]:
prompt = """
Today is Monday, tomorrow is Wednesday.

What is wrong with this sentence?
"""

In [ ]:
textgen_llm(prompt)

**Prompt template**

In [ ]:
from langchain import PromptTemplate

In [ ]:
template = """
I want to travel to {location}. What should I do there?

Respond in one short sentence.
"""

In [ ]:
prompt = PromptTemplate(
    input_variables=["location"],
    template=template,
)

final_prompt = prompt.format(location="Munich")

In [ ]:
print(f"Final prompt:{final_prompt}")
print("-------------")

print(f"LLM response: {textgen_llm(final_prompt)}")

<h3>Output Parsers</h3>

To format the output of a model.
1. Format instructions
2. Parser

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [ ]:
# How you would like your response structured. This is basically a fancy prompt template

response_schemas = [
    ResponseSchema(name="user_input", description="This the users input"),
    ResponseSchema(name="emoji", description="This is your response, a reformatted response only with the emoji"),
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


In [ ]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
template = """
You will be given a user input.
extract the emoji that is inside it and answer back only with the emoji.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="I am playing football")

print(promptValue)

In [ ]:
output = textgen_llm(promptValue)
print(output)

In [ ]:
textgen_llm(promptValue)

### Document loaders

Langchain document loaders: https://python.langchain.com/docs/modules/data_connection/document_loaders/

**PDF**

In [ ]:
!pip install pypdf

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("AMZN-2022-Shareholder-Letter.pdf")
data = loader.load()

In [ ]:
print(data)

**Web scrapping**

In [ ]:
!pip install unstructured

In [ ]:
url = ['https://www.espn.com/']

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(url)
data = loader.load()

In [ ]:
print(data)

<h3> Chains </h3> 
https://python.langchain.com/docs/modules/chains/

**Simple sequential chains**

In [ ]:
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain

In [ ]:
template = """
your job is to come up with a classic dish from the area that the users suggests. The name of the dish has to be in English.
% USER LOCATION:
{user_location}

%YOUR RESPONSE:
"""

prompt_template = PromptTemplate(template=template, input_variables=['user_location'])

location_chain = LLMChain(llm=textgen_llm, prompt=prompt_template)

In [ ]:
template = """
Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL:
{user_meal}

% YOUR RESPONSE:
"""

promt_template = PromptTemplate(template=template, input_variables=['user_meal'])

meal_chain = LLMChain(llm=textgen_llm, prompt=promt_template)

In [ ]:
overal_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)

In [ ]:
review = overal_chain.run("Paris")

## Retrieval Augmented Generation (RAG)

Langchain RAG: https://python.langchain.com/docs/use_cases/question_answering/

### Data loader

In [ ]:
url = ['https://www.barcelonaturisme.com/wv3/en/']

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(url)
data = loader.load()

### Text Splitter

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator='\n', 
                                      chunk_size=2000, 
                                      chunk_overlap=200)


docs = text_splitter.split_documents(data)

### Embedding model

In [ ]:
from langchain.embeddings import BedrockEmbeddings

bedrock_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1",
    client=boto3_bedrock
)

### Vector database

In [ ]:
import pickle
import faiss
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

embeddings = bedrock_embeddings

In [ ]:
docsearch = FAISS.from_documents(data, embeddings)

### RAG Chain

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """

Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer from the context, just say you don't know, don't try to make up an answer.

{context}

Question: {question}

Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

qa = RetrievalQA.from_chain_type(llm=textgen_llm,chain_type="stuff",retriever=docsearch.as_retriever(),return_source_documents=True,chain_type_kwargs={"prompt": PROMPT},
)

In [ ]:
result = qa({"query": "What can I do in Barcelona?"})
print_ww(result["result"])

In [ ]:
result = qa({"query": "What are the monthly events in Rome this month?"})
print_ww(result["result"])

## Langchain advanced

<h3> Chat History Basic</h3>

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [ ]:
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=textgen_llm, verbose=True, memory=memory
)

In [ ]:
print_ww(conversation.predict(input="which countries can you recommend to visit?"))

In [ ]:
memory.clear()

### Chat history dynamodb 

In [ ]:
dynamodb = boto3.resource("dynamodb")

# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName="SessionTable",
    KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
    AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
    BillingMode="PAY_PER_REQUEST",
)

# Wait until the table exists.
table.meta.client.get_waiter("table_exists").wait(TableName="SessionTable")

# Print out some data about the table.
print(table.item_count)

In [ ]:
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory

memory_db1 = DynamoDBChatMessageHistory(table_name="SessionTable", session_id="4")

memory_db2 = DynamoDBChatMessageHistory(table_name="SessionTable", session_id="5")

memory1 = ConversationBufferMemory(
    memory_key="history", chat_memory=memory_db1, return_messages=True
)

memory2 = ConversationBufferMemory(
    memory_key="history", chat_memory=memory_db2, return_messages=True
)


In [ ]:
conversation = ConversationChain(
    llm=textgen_llm, verbose=True, memory=memory2
)

print_ww(conversation.predict(input="Hello"))

<h3> Agents </h3>

You use the LLM not just for text output but for decision making. 1. Agent, 2. Tool (like plugins), 3.Toolkit (colletion of plugins)

list of tools: https://python.langchain.com/docs/integrations/tools

In [ ]:
os.environ["SERPAPI_API_KEY"] = "71c02d43a64cae1b3a07a3dc16b6abb3dcd9fe98f55a99680419509a90255552"

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms.bedrock import Bedrock
from langchain import LLMMathChain
from langchain.utilities import SerpAPIWrapper

In [ ]:
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}
react_agent_llm = Bedrock(model_id="anthropic.claude-instant-v1", model_kwargs=model_parameter)
math_chain_llm = Bedrock(model_id="anthropic.claude-instant-v1",
                         model_kwargs={"temperature":0,"stop_sequences" : ["```output"]})

tools = load_tools(["serpapi"], llm=react_agent_llm)

llm_math_chain = LLMMathChain(llm=math_chain_llm, verbose=True)

llm_math_chain.llm_chain.prompt.template = """Human: Given a question with a math problem, provide only a single line mathematical expression that solves the problem in the following format. Don't solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Assistant:
 Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math.",
    )
)

In [ ]:
react_agent = initialize_agent(tools, 
                               react_agent_llm, 
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                               verbose=True,
                            #    max_iteration=2,
                            #    return_intermediate_steps=True,
                            #    handle_parsing_errors=True,
                               )

In [ ]:
prompt_template = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of ["Search", "Calculator"]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}"""

In [ ]:
react_agent.agent.llm_chain.prompt.template=prompt_template

In [ ]:
question = "What is Amazon Sagemaker? What is the launch year multiplied by 2"

In [ ]:
react_agent(question)

### Query agents

In [ ]:
llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)

In [ ]:
customer_table=[
  {
    "id": 1, 
    "first_name": "John", 
    "last_name": "Doe",
    "age": 35,
    "postal_code": "90210"
  },
  {  
    "id": 2,
    "first_name": "Jane",
    "last_name": "Smith", 
    "age": 27,
    "postal_code": "12345"
  },
  {
    "id": 3, 
    "first_name": "Bob",
    "last_name": "Jones",
    "age": 42,
    "postal_code": "55555"
  },
  {
    "id": 4,
    "first_name": "Sara", 
    "last_name": "Miller",
    "age": 29, 
    "postal_code": "13579"
  },
  {
    "id": 5,
    "first_name": "Mark",
    "last_name": "Davis",
    "age": 31,
    "postal_code": "02468"
  },
  {
    "id": 6,
    "first_name": "Laura",
    "last_name": "Wilson",
    "age": 24,
    "postal_code": "98765" 
  },
  {
    "id": 7,
    "first_name": "Steve",
    "last_name": "Moore",
    "age": 36,
    "postal_code": "11223"
  },
  {
    "id": 8,
    "first_name": "Michelle",
    "last_name": "Chen",
    "age": 22,
    "orders": [
        {
            "order_id": 1,
            "description": "An order of 1 dozen pencils"
        },
        {
            "order_id": 2,
            "description": "An order of 2 markers"
        }
    ],
    "postal_code": "33215"
  },
  {
    "id": 9,
    "first_name": "David",
    "last_name": "Lee",
    "age": 29,
    "postal_code": "99567"
  },
  {
    "id": 10,
    "first_name": "Jessica",
    "last_name": "Brown",
    "age": 18, 
    "postal_code": "43210"
  }
]

def customer_lookup(id):
    print(f"search by customer {id}")
    for customer in customer_table:
        if customer["id"] == int(id):
            print(f"found customer {id} {customer}")
            return customer
        
    return None

tools.append(Tool.from_function(
        name="CustomerLookup",
        func=customer_lookup,  # Mock Function, replace with an api call
        description="Use this when you need to lookup a customer by id."
    ))

In [ ]:
react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """\n\nHuman: write one sentence summary about the information you know about the customer with an id of 5.

\n\nAssistant: Here is the one sentence summary: """

result = react_agent.run(question)

print(f"{result}")